In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from lm_polygraph.utils.model import WhiteboxModel, BlackboxModel
from lm_polygraph.utils.manager import estimate_uncertainty
from lm_polygraph.estimators import MaximumTokenProbability, LexicalSimilarity, SemanticEntropy, PointwiseMutualInformation, EigValLaplacian,MeanPointwiseMutualInformation,SAR,SentenceSAR,MaximumSequenceProbability,TokenSAR
from tqdm.notebook import tqdm
tqdm.pandas(ProgressBar=True)

## Selected low-computational estimator


In [ ]:
LexicalSimilarity
PointwiseMutualInformation
MaximumSequenceProbability

## Selected high-computational estimator

In [ ]:
SAR
SemanticEntropy

In [44]:

from lm_polygraph.estimators import LexicalSimilarity

# Put your 5-10 text samples from ChatGPT generation
# samples = [
#     'The capital of France is Paris.',
#     'Paris is the capital city of France.',
#     'The capital of France is Paris.',
#     'In France, the capital is Paris.',
#     'The capital city of France is Paris.',
# ]

samples = [
    
    'Paris is the capital city of France.',
    'The capital of France is Paris.',
    'In France, the capital is Paris.',
    'The capital city of France is Paris.',
    'The capital of France is Paris.',
]

estimator = LexicalSimilarity('rougeL')
uncertainty = estimator({
    'blackbox_sample_texts': [samples],
})
print(uncertainty[1])  # -0.7047619047619047

[-0.61538462 -0.30769231 -0.71428571 -0.61538462 -0.66666667 -0.92307692
 -1.         -0.61538462 -0.66666667 -0.92307692]


In [1]:
import numpy as np
import torch
from lm_polygraph.estimators import LexicalSimilarity
# stats = np.load('../lm-polygraph/examples/sent_sar_1726031992.7997897.npy',allow_pickle=True).item()
# Put your 5-10 text samples from ChatGPT generation
# samples = [
#     'The capital of France is Paris.',
#     'Paris is the capital city of France.',
#     'The capital of France is Paris.',
#     'In France, the capital is Paris.',
#     'The capital city of France is Paris.',
# ]

samples = [
    [
    
    'Paris is the capital city of France.',
    'The capital of France is Paris.',
    'In France, the capital is Paris.',
    'The capital city of France is Paris.',
    'The capital of France is Paris.',
],
    [
    'The capital of France is Paris.',
    'Paris is the capital city of France.',
    'The capital of France is Paris.',
    'In France, the capital is Paris.',
    'The capital city of France is Paris.',
]
    ]

estimator = SentenceSAR()
uncertainty = estimator()
# print(uncertainty)  # -0.7047619047619047
# uncertainty = estimator(stats[1])
# print(uncertainty)  # -0.7047619047619047

/home/wangys/anaconda3/envs/deepspeed/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


NameError: name 'SentenceSAR' is not defined

In [67]:
# uncertainty[1].argmin()
np.sum(stats['sample_sentence_similarity'][0],axis=1)

array([3.6768064, 3.7241166, 3.7855449, 3.6245348, 3.7432592, 3.8125243,
       3.628025 , 3.6888838, 3.7558572, 3.7589967], dtype=float32)

In [61]:
# 
stats['sample_texts'][0][uncertainty[1].argmin()]

'Barack Obama is a renowned American politician who served as the 44th President of the United States from 2009 to 2017. He was the first African American to hold the office. Obama was born on August 4, 1961, in Honolulu, Hawaii, and raised in Hawaii and Indonesia. He attended Columbia University in New York City and later earned his law degree from Harvard Law School.\n\nBefore his presidency,'

In [12]:
from lm_polygraph.estimators import Eccentricity
from lm_polygraph.stat_calculators import SemanticMatrixCalculator
from lm_polygraph.utils import Deberta
# Put your 5-10 text samples from ChatGPT generation
samples = [
    'The capital of France is Paris.',
    'Paris is the capital city of France.',
    'The capital of France is Paris.',
    'In France, the capital is Paris.',
    'The capital city of France is Paris.',
]

stats = {
    'blackbox_sample_texts': [samples]
}
# model = deberta()
nli_calculator = SemanticMatrixCalculator(nli_model=Deberta)
stats.update(nli_calculator(stats, None, None, None))
# Now stats should contain 'semantic_matrix_entail' and 'semantic_matrix_contra'

estimator = Eccentricity()
uncertainty = estimator(stats)[0]
print(uncertainty)  # 7.886122580038351e-05

ImportError: cannot import name 'Deberta' from 'lm_polygraph.utils' (/home/wangys/anaconda3/envs/deepspeed/lib/python3.12/site-packages/lm_polygraph/utils/__init__.py)

In [13]:
import numpy as np

## From Here We Extract Enriched Data

In [291]:
query = np.load('../Transfer-ER-Blocking/enrich_data/enrich_query_UE/AG_output_Mistral-7B-Instruct-v0.2_logprob.npy',allow_pickle=True).item()

## 解析结果

In [69]:
def extract_and_wrap(text):
    # 查找第一个 '{' 和最后一个 '}' 的索引
    start_index = text.find('{')
    end_index = text.rfind('}')

    # 检查是否有匹配的 '{' 和 '}'
    if start_index == -1 or end_index == -1 or start_index > end_index:
        return "No matching braces found."

    # 提取第一个 '{' 和最后一个 '}' 之间的内容
    extracted_content = text[start_index + 1:end_index]

    # 在头部和尾部分别补上 '{' 和 '}'
    wrapped_content = "{" + extracted_content + "}"

    return wrapped_content
def extract_and_wrap_multi_dict(text):
    # 查找第一个 '{' 和最后一个 '}' 的索引
    start_index = text.rfind('\n\n{"Entity 1')
    end_index = text.rfind('}')

    # 检查是否有匹配的 '{' 和 '}'
    if start_index == -1 or end_index == -1 or start_index > end_index:
        return "No matching braces found."

    # 提取第一个 '{' 和最后一个 '}' 之间的内容
    extracted_content = text[start_index + 1:end_index]

    # 在头部和尾部分别补上 '{' 和 '}'
    wrapped_content = extracted_content + "}"

    return wrapped_content

In [71]:
data_name = 'AG'

enrich_query_path = '../Transfer-ER-Blocking/enrich_data/enrich_query/%s_input.csv' % data_name


enrich_query = pd.read_csv(enrich_query_path,index_col=0)
enrich_query['ltable_enrich'] = ''
enrich_query['rtable_enrich'] = ''

In [304]:

from json_repair import repair_json
import json
count = 0
LLM_output = query['text']
dict_ltable = {}
dict_rtable = {}
for i in tqdm(range(len(enrich_query))):
    ltable_id = int(enrich_query.iloc[i,0])
    rtable_id = int(enrich_query.iloc[i,1])
    if not dict_ltable.__contains__(ltable_id):
        dict_ltable[ltable_id] = []
    if not dict_rtable.__contains__(rtable_id):
        dict_rtable[rtable_id] = [] ## 初始化
# for i in tqdm(range(len(LLM_output))):
    text = LLM_output[i]
    if(text.__contains__('json')): ## 有```json```,只看```json包裹内的内容
        a = repair_json(text.split('```json')[-1].split('```')[0])
        temp = text.split('```json')[-1].split('```')[0]
        # print(temp)
        # print('str 1')
    else:
        a = repair_json(extract_and_wrap(text)) ##匹配第一个{与最后一个}
        # print('str 2')
    try:
        dict_ouput = json.loads(a)
        enrich_query.iloc[i,-2] = str(dict_ouput['Entity 1'])
        enrich_query.iloc[i,-1] = str(dict_ouput['Entity 2'])
        dict_ouput['Entity 1']
        # print('case 1')
    except:
        try:
            a = repair_json(extract_and_wrap_multi_dict(text)) ## 匹配最后一个\n\n{"Entity 1 与最后一个'
            dict_ouput = json.loads(a)
            enrich_query.iloc[i,-2] = str(dict_ouput['Entity 1'])
            enrich_query.iloc[i,-1] = str(dict_ouput['Entity 2'])
            # print('case 2')
        except:
            count += 1
    token_list = query['token'][i]
    logprob_list = query['logprob'][i]
    ## 开始Search，从左到右
    # if(isinstance(dict_ouput,dict)): ## 检查dict_ouput是不是dict，这是一种检查方法
    if(dict_ouput.__contains__('Entity 1') and dict_ouput.__contains__('Entity 2')):
        start_place = 0
        for key in list(dict_ouput['Entity 1'].keys()): ## 跳过原有属性n个,n=3
            log_prob = []
            element = dict_ouput['Entity 1'][key]
            if not element=='': ##不是空字符
                encoded_element = tokenizer.encode('"{}'.format(element),add_special_tokens=False) ## 补充元素必定以"，也就是token:345开始，切换tokenizer要对应切换
                encoded_token_len = len(encoded_element)
                search_place = [token_start for token_start in find_sublist_positions(encoded_element,token_list) if token_start>=start_place] ## 搜索位置大于start_place
                if(search_place!=[]): ## 搜索不到
                    # print(len(search_place))
                    start_place = search_place[0]
                    for token_start in search_place:
                        prob_sequence = logprob_list[token_start+1:token_start+encoded_token_len]
                        log_prob.append(prob_sequence)
            dict_ouput['Entity 1']['{}_logprob'.format(key)] = log_prob
        dict_ltable[ltable_id].append(dict_ouput['Entity 1']) ## 写入attribute-level logprob
        for key in list(dict_ouput['Entity 2'].keys()): ## 跳过原有属性n个,n=3
            log_prob = []
            element = dict_ouput['Entity 2'][key]
            if not element=='': ##不是空字符
                encoded_element = tokenizer.encode('"{}'.format(element),add_special_tokens=False) ## 补充元素必定以"，也就是token:345开始，切换tokenizer要对应切换
                encoded_token_len = len(encoded_element)
                search_place = [token_start for token_start in find_sublist_positions(encoded_element,token_list) if token_start>=start_place] ## 搜索位置大于start_place
                if(search_place!=[]): ## 搜索不到
                    # print(len(search_place))
                    start_place = search_place[0]
                    for token_start in search_place:
                        prob_sequence = logprob_list[token_start+1:token_start+encoded_token_len]
                        log_prob.append(prob_sequence)
            dict_ouput['Entity 2']['{}_logprob'.format(key)] = log_prob
        dict_rtable[rtable_id].append(dict_ouput['Entity 2'])

  0%|          | 0/11460 [00:00<?, ?it/s]

In [5]:
import numpy as np
import itertools
from sentence_transformers import CrossEncoder
from transformers import AutoTokenizer
from FlagEmbedding import FlagReranker
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
stats = np.load('/data/home/wangys/lm-polygraph/examples/sar.npy',allow_pickle=True).item()

def setup_cross_encoder(device='cuda', model_path = '/data/home/wangys/sentence_transformer_model/stsb-roberta-base'):
    cross_encoder = CrossEncoder(model_path, device=device)
    return cross_encoder

def setup_reranker(device='cuda', model_path = '/data/home/wangys/sentence_transformer_model/bge-rerank-large'):
    reranker = FlagReranker(model_path, use_fp16=True)
    return reranker

# def calculate_sample_sentence_similarity(dependencies, cross_encoder):
#     batch_texts = dependencies["sample_texts"]
#     batch_pairs = []
#     batch_invs = []
#     batch_counts = []
    
#     for texts in batch_texts:
#         unique_texts, inv = np.unique(texts, return_inverse=True)
#         batch_pairs.append(list(itertools.product(unique_texts, unique_texts)))
#         batch_invs.append(inv)
#         batch_counts.append(len(unique_texts))

#     sim_matrices = []
#     for i, pairs in enumerate(batch_pairs):
#         sim_scores = cross_encoder.predict(pairs)
#         # sim_scores = cross_encoder.compute_score(pairs)
#         unique_mat_shape = (batch_counts[i], batch_counts[i])
#         sim_scores_matrix = sim_scores.reshape(unique_mat_shape)
#         inv = batch_invs[i]
#         sim_matrices.append(sim_scores_matrix[inv, :][:, inv])
        
#     sim_matrices = np.stack(sim_matrices)
#     return sim_matrices

def calculate_sample_sentence_similarity_reranker(dependencies, cross_encoder):
    batch_texts = dependencies["sample_texts"]
    batch_pairs = []
    batch_invs = []
    batch_counts = []
    
    for texts in batch_texts:
        unique_texts, inv = np.unique(texts, return_inverse=True)
        batch_pairs.append(list(itertools.product(unique_texts, unique_texts)))
        batch_invs.append(inv)
        batch_counts.append(len(unique_texts))

    sim_matrices = []
    for i, pairs in enumerate(batch_pairs):
        # sim_scores = cross_encoder.predict(pairs)
        sim_scores = np.array(cross_encoder.compute_score(pairs))
        unique_mat_shape = (batch_counts[i], batch_counts[i])
        sim_scores_matrix = sim_scores.reshape(unique_mat_shape)
        inv = batch_invs[i]
        sim_matrices.append(sim_scores_matrix[inv, :][:, inv])
        
    sim_matrices = np.stack(sim_matrices)
    return sim_matrices

# Example usage in a Jupyter Notebook Cell
device = 'cuda:0'  # or 'cpu'
# tokenizer = AutoTokenizer.from_pretrained("/data/home/wangys/sentence_transformer_model/stsb-roberta-base")
# cross_encoder = setup_cross_encoder(device=device)
# tokenizer = AutoTokenizer.from_pretrained("/data/home/wangys/sentence_transformer_model/bge-rerank-large/")

# cross_encoder = setup_reranker(device=device)

# Assuming dependencies is a dictionary containing 'sample_texts' key
# For example:
# dependencies = {'sample_texts': [['Hello world', 'Hello world', 'Goodbye world'], ['Sample text 1', 'Sample text 2']]}
# dependencies = {
#     'sample_texts': [
#         ['Hello world', 'Goodbye world'],
#         ['Sample text 1', 'Sample text 2']
#     ]
# }

sample_sentence_similarity = calculate_sample_sentence_similarity_reranker(stats, cross_encoder)
print(sample_sentence_similarity)

[[[9.5234375  1.87207031 2.3671875  2.39648438 2.30859375 2.9765625
   2.51953125 3.47460938 1.83886719 5.55859375]
  [2.38867188 9.53125    5.37890625 6.28515625 3.89257812 5.21484375
   7.203125   5.10546875 5.6484375  7.71875   ]
  [2.6953125  7.796875   9.5234375  9.5390625  6.30859375 8.375
   9.515625   9.34375    9.5        7.375     ]
  [4.37890625 5.9609375  9.3515625  9.5390625  6.5234375  9.015625
   9.3515625  9.3671875  8.3046875  6.4765625 ]
  [2.80859375 4.2890625  6.90234375 7.64453125 9.5390625  7.8828125
   7.28125    6.44140625 6.70703125 8.875     ]
  [2.15820312 6.07421875 8.0859375  8.265625   9.03125    9.5234375
   9.3515625  8.         8.1015625  9.46875   ]
  [2.74804688 5.75390625 9.5390625  9.5390625  6.98828125 9.40625
   9.53125    9.53125    9.53125    6.390625  ]
  [3.44726562 6.546875   9.2734375  9.515625   6.01171875 8.546875
   9.453125   9.5234375  8.8984375  9.234375  ]
  [1.58007812 5.93359375 9.2109375  8.8984375  7.203125   9.0625
   9.4296875  

/home/wangys/anaconda3/envs/deepspeed/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [323]:
import numpy as np
import itertools
from sentence_transformers import CrossEncoder
from transformers import AutoTokenizer

def setup_cross_encoder(device='cuda',path='/data/home/wangys/sentence_transformer_model/stsb-roberta-base'):
    cross_encoder = CrossEncoder(path, device=device)
    return cross_encoder

def calculate_similarities(dependencies, cross_encoder, tokenizer):
    batch_input_texts = dependencies["input_texts"]
    batch_greedy_tokens = dependencies["greedy_tokens"]
    batch_sample_tokens = dependencies["sample_tokens"]
    batch_texts = dependencies["sample_texts"]
    
    special_tokens = list(tokenizer.all_special_tokens)

    # Calculate sample_sentence_similarity
    batch_pairs = []
    batch_invs = []
    batch_counts = []
    for texts in batch_texts:
        unique_texts, inv = np.unique(texts, return_inverse=True)
        batch_pairs.append(list(itertools.product(unique_texts, unique_texts)))
        batch_invs.append(inv)
        batch_counts.append(len(unique_texts))

    sim_matrices = []
    for i, pairs in enumerate(batch_pairs):
        sim_scores = cross_encoder.predict(pairs)
        unique_mat_shape = (batch_counts[i], batch_counts[i])
        sim_scores_matrix = sim_scores.reshape(unique_mat_shape)
        inv = batch_invs[i]
        sim_matrices.append(sim_scores_matrix[inv, :][:, inv])
    sim_matrices = np.stack(sim_matrices)

    # Calculate token_similarity
    batch_token_scores = []
    for input_texts, tokens in zip(batch_input_texts, batch_greedy_tokens):
        if len(tokens) > 1:
            is_special_tokens = np.isin(tokens, special_tokens)
            cropped_tokens = list(itertools.combinations(tokens, len(tokens) - 1))[::-1]
            raw_text = input_texts + " " + tokenizer.decode(tokens, skip_special_tokens=True)
            batches = [
                (raw_text, input_texts + " " + tokenizer.decode(list(t), skip_special_tokens=True))
                for t in cropped_tokens
            ]
            token_scores = cross_encoder.predict(batches)
            token_scores[is_special_tokens] = 1
        else:
            token_scores = np.array([0.5] * len(tokens))
        batch_token_scores.append(token_scores)

    # Calculate sample_token_similarity
    batch_samples_token_scores = []
    for sample_tokens, input_texts in zip(batch_sample_tokens, batch_input_texts):
        samples_token_scores = []
        for tokens in sample_tokens:
            if len(tokens) > 1:
                is_special_tokens = np.isin(tokens, special_tokens)
                cropped_tokens = list(itertools.combinations(tokens, len(tokens) - 1))[::-1]
                raw_text = input_texts + " " + tokenizer.decode(tokens, skip_special_tokens=True)
                batches = [
                    (raw_text, input_texts + " " + tokenizer.decode(list(t), skip_special_tokens=True))
                    for t in cropped_tokens
                ]
                token_scores = cross_encoder.predict(batches)
                token_scores[is_special_tokens] = 1
            else:
                token_scores = np.array([0.5] * len(tokens))
            samples_token_scores.append(token_scores)
        batch_samples_token_scores.append(samples_token_scores)

    return {
        "sample_sentence_similarity": sim_matrices,
        "sample_token_similarity": batch_samples_token_scores,
        "token_similarity": batch_token_scores,
    }

# Example usage in a Jupyter Notebook Cell
device = 'cuda'  # or 'cpu'
path = '/data/home/wangys/sentence_transformer_model/stsb-roberta-base'
tokenizer = AutoTokenizer.from_pretrained(path)
cross_encoder = setup_cross_encoder(device=device,path=path)

# Assuming dependencies is a dictionary containing 'input_texts', 'greedy_tokens', 'sample_tokens', and 'sample_texts'
# dependencies = {
#     'input_texts': ['Hello world', 'Another example text'],
#     'greedy_tokens': [[101, 7592, 2088, 102], [101, 2178, 1859, 3793, 102]],  # Token IDs
#     'sample_tokens': [[[101, 7592, 2088, 102], [101, 7592, 102]], [[101, 2178, 1859, 3793, 102]]],
#     'sample_texts': [['Hello world', 'Hi world'], ['Another example text']],
# }

results = calculate_similarities(stats, cross_encoder, tokenizer)
print(results['sample_sentence_similarity'])

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /data/home/wangys/sentence_transformer_model/stsb-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[[[0.5596442  0.5714566  0.5639174  0.55540067 0.5599863  0.5592013
   0.5633068  0.55737704 0.5523158  0.5403635 ]
  [0.56727153 0.5606382  0.56034434 0.5532132  0.568062   0.5576059
   0.5560436  0.5576421  0.5453054  0.5368302 ]
  [0.55708975 0.5598821  0.55567086 0.5485729  0.55802745 0.55198365
   0.55195445 0.5495054  0.5401506  0.5239375 ]
  [0.549484   0.55294096 0.5471738  0.53400093 0.55239207 0.5420458
   0.5456812  0.5407787  0.5280566  0.5178532 ]
  [0.5608775  0.5643278  0.5583287  0.5541518  0.55305284 0.55056393
   0.55016094 0.5579378  0.54632753 0.5296941 ]
  [0.55421287 0.55401003 0.5453452  0.5433912  0.5511032  0.53545564
   0.5367594  0.54644996 0.53402567 0.52085215]
  [0.5609234  0.5604612  0.5529131  0.5482103  0.5540797  0.54461753
   0.54455215 0.5512292  0.53674155 0.5267632 ]
  [0.5486426  0.55080146 0.54625505 0.5391846  0.5501874  0.5425313
   0.54158264 0.5394819  0.5292997  0.5160477 ]
  [0.53878725 0.5356601  0.5285249  0.5225351  0.5416198  0.5268245


In [307]:
np.save('../Transfer-ER-Blocking/enrich_data/enrich_dict/dict_ltable_AG_logprob.npy',dict_ltable)
np.save('../Transfer-ER-Blocking/enrich_data/enrich_dict/dict_rtable_AG_logprob.npy',dict_rtable)

In [266]:
a = pd.read_csv('../Transfer-ER-Blocking/enrich_data/enrich_query/AG_input.csv',index_col=0)
a.iloc[0,-1]

'Enrich Entity 1 and Entity 2 with attributes: category/subcategory/platform/edition/type/modelno. Return in json format.\n\nOutput Format Example:\n\n{"Entity 1": {"title": "", "manufacturer": "", "price": "", "category": "", "subcategory": "", "platform": "", "edition": "", "type": "", "modelno": ""}, "Entity 2": {"title": "", "manufacturer": "", "price": "", "category": "", "subcategory": "", "platform": "", "edition": "", "type": "", "modelno": ""}}\n\nEntity 1:{"title": "microsoft visio standard 2007 version upgrade", "manufacturer": "microsoft", "price": 129.95}\n\nEntity 2:{"title": "adobe cs3 design standard upgrade", "manufacturer": "", "price": 413.99}'

In [74]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('../model/Mistral-7B-Instruct-v0.2/')

[345,  8684,  1264,   345] caterogy
[345,  1666,  8684,  1264,   345] sub-caterogy
[345, 10470,  1264,   345] platform
[345,   286,   685,  1264,   345] edition
[345,  3549,  1510,  1264,   345] modelno

In [ ]:
def find_sublist_positions(sublist, mainlist):
    positions = []
    len_sublist = len(sublist)
    # 遍历可能的起始位置
    for i in range(len(mainlist) - len_sublist + 1):
        # 判断从i开始的子列表是否与sublist相等
        if mainlist[i:i + len_sublist] == sublist:
            positions.append(i)
    return positions
category = [345,  8684,  1264,   345]
sub_cat = [345,  1666,  8684,  1264,   345]
platform = [345, 10470,  1264,   345]
edition = [345,   286,   685,  1264,   345]
modelno = [345,  3549,  1510,  1264,   345]
for i in range(500):
    # print(find_sublist_positions([345,8684],query['token'][i]))
    sublist_all = [category,sub_cat,platform]
    for sublist in sublist_all:
        result = find_sublist_positions(sublist,query['token'][i])
        if(len(result)!=2):
            print(i,tokenizer.decode(sublist))

In [182]:
# def find_concat_positions_optimized(lst, input_str):
def find_concat_positions_optimized(lst, input_str):
    positions = []
    n = len(input_str)
    len_lst = len(lst)
    # 创建一个集合，包含input_str中的所有字符
    input_str_chars = set(input_str)
    
    # 遍历列表中的每个可能的起始点
    for i in range(len_lst):
        # 检查当前元素是否为空，以及首字符是否在input_str的字符集中
        if lst[i] and lst[i][0] in input_str_chars:
            # 拼接从i开始的足够多的元素以达到input_str的长度
            concatenated_str = ''.join(lst[i:i+n])
            # 如果拼接后的字符串与input_str相等，记录起始位置
            if concatenated_str == input_str:
                positions.append(i)
    
    return positions
find_concat_positions_optimized(query['decoded_token'][0][110:150],'Software')

[]

In [198]:
# def find_concat_positions_flexible(lst, input_str):
#     positions = []
#     n = len(input_str)
#     len_lst = len(lst)
    
#     # 遍历列表中的每个可能的起始点
#     for i in range(len_lst):
#         concatenated_str = ''  # 初始化拼接字符串
#         for j in range(i, len_lst):
#             concatenated_str += lst[j].strip()  # 拼接时去除两端空格
#             # 检查当前拼接字符串是否与input_str匹配
#             if concatenated_str == input_str:
#                 positions.append((i, j))  # 记录起始和结束索引
#                 break  # 匹配成功后不需要继续拼接
#             elif len(concatenated_str) > n:
#                 break  # 如果拼接长度已经超过input_str的长度，不需要继续拼接

#     return positions

def find_concat_positions_flexible(lst, input_str):
    positions = []
    len_lst = len(lst)
    
    # 遍历列表中的每个可能的起始点
    for i in range(len_lst):
        concatenated_str = ''  # 初始化拼接字符串
        concat_positions = []  # 用于记录参与拼接的元素索引
        for j in range(i, len_lst):
            # 将当前元素加入拼接，并去除两端的空格
            current_piece = lst[j].strip()
            if current_piece:  # 只有非空的元素才参与拼接
                concatenated_str += current_piece
                concat_positions.append(j)
                # 检查当前拼接字符串是否与input_str匹配
                if concatenated_str == input_str:
                    positions.append((concat_positions[0], concat_positions[-1]))  # 记录起始和结束索引
                    break  # 匹配成功后不需要继续拼接
                elif len(concatenated_str) > len(input_str):
                    break  # 如果拼接长度已经超过input_str的长度，不需要继续拼接

    return positions

# 示例
lst = [',', '\n', '   ', ' "', 'category', '":', ' "', 'Soft', 'ware', '",', '\n', '   ', ' "', 'sub', 'category', '":', ' "', 'Product', 'ivity', ' Software', '",', '\n', '   ', ' "', 'platform', '":', ' "', 'Windows', '",', '\n', '   ', ' "', 'ed', 'ition', '":', ' "', 'Standard', '",', '\n', '   ']
input_str = 'Productivity Software'
positions = find_concat_positions_flexible(query['decoded_token'][0], input_str)
print(positions)  # 输出应该是 [(7, 8)]

[]


In [218]:
tokenizer.encode('"CS3DESIGNUPG',add_special_tokens=False)

[345, 6158, 28770, 1336, 20506, 3571, 28777]

encode需要加单边"+内容不加另一边的"，如'"CS3DESIGNUPG'

In [219]:
find_sublist_positions([345, 6158, 28770, 1336, 20506, 3571, 28777],query['token'][0])

[275]

In [227]:
# query['text'][423]
tokenizer.encode('"Entity 1',add_special_tokens=False),tokenizer.encode('"Entity 2',add_special_tokens=False)

([345, 5075, 28705, 28740], [345, 5075, 28705, 28750])

In [228]:
search = dict_ouput['Entity 1']['platform']
find_sublist_positions(tokenizer.encode('"{}'.format(search),add_special_tokens=False),query['token'][0])
find_sublist_positions([345, 5075, 28705, 28740],query['token'][0])

[63]

## 从左到右搜索


In [244]:
query['token'][4][1:4]

[5075, 28705, 28740]

In [249]:
# tokenizer.decode([5075, 28705, 28740])
tokenizer.encode('"Entity 1')

[1, 345, 5075, 28705, 28740]

In [258]:
# query['text'][259]
token_example = '"Education'
find_sublist_positions(tokenizer.encode(token_example,add_special_tokens=False),query['token'][259])
# tokenizer.decode(query['token'][4][1:4])

[233]

In [254]:
entity_1 = tokenizer.encode('"Entity 1',add_special_tokens=False) ## Entity 1指针开始位置
# entity_1_whole = tokenizer.encode('Entity 1":',add_special_tokens=False) ## Entity 1指针开始位置
entity_2 = tokenizer.encode('"Entity 2',add_special_tokens=False) ## Entity 2指针开始位置
for i in range(500):
    # if find_sublist_positions(entity_1,query['token'][i])==[] and find_sublist_positions(entity_1[1:],query['token'][i])==[]: ## 找不到"Entity 1,可能因为左"被忽略了，去掉token:345
    #     print(i)
    # if find_sublist_positions(entity_2,query['token'][i])==[] and find_sublist_positions(entity_2[1:],query['token'][i])==[]: ## 找不到"Entity 1：
    #     print(i)
    result = find_sublist_positions(entity_1[1:],query['token'][i])
    if(len(result)!=1):
        print(i,result)


10 [51, 195]
19 [54, 275, 293]
21 [52, 158]
34 [54, 259]
41 [55, 186]
43 [52, 210]
64 [52, 179]
66 [51, 209]
73 [53, 197]
90 [52, 173]
92 [53, 186]
94 [52, 212]
96 [53, 168]
124 [52, 203]
128 [53, 189]
142 [53, 168]
147 [30, 240]
176 [51, 174]
178 [35, 310]
193 [52, 160]
201 [53, 210]
209 [25, 75]
217 [55, 206]
224 [35, 160]
225 [52, 143]
240 [52, 172]
244 [52, 193]
248 [53, 156]
259 [25, 76]
270 [51, 174]
283 [51, 179]
308 [53, 170]
324 [25, 78]
334 [53, 194]
336 [53, 189]
339 [52, 173]
340 [87, 316]
347 [53, 193]
349 [35, 137]
357 [52, 122]
362 [52, 184]
367 [52, 144]
373 [53, 202]
375 [32, 150]
398 [54, 196]
404 [54, 255]
406 [53, 186]
415 [52, 180]
417 [55, 196]
450 [53, 171]
456 [52, 186]
470 [60, 291, 309]
475 [52, 175]
480 [53, 188]
493 [52, 199]


In [201]:
def find_concat_positions_flexible(lst, input_str):
    positions = []
    len_lst = len(lst)
    
    # 遍历列表中的每个可能的起始点
    for i in range(len_lst):
        concatenated_str = ''  # 初始化拼接字符串
        concat_positions = []  # 用于记录参与拼接的元素索引
        for j in range(i, len_lst):
            current_piece = lst[j].strip()  # 去除当前字符串两端空格
            if current_piece:  # 只处理非空字符串
                # 如果拼接后的字符串仍然不超过目标字符串，才继续拼接
                if len(concatenated_str + current_piece) <= len(input_str):
                    concatenated_str += current_piece
                    concat_positions.append(j)
                    # 完全匹配后返回结果
                    if concatenated_str == input_str:
                        positions.append((concat_positions[0], concat_positions[-1]))
                        break
                else:
                    break  # 超出字符串长度，中断内层循环

    return positions

# 示例
lst = [',', '\n', '   ', ' "', 'category', '":', ' "', 'Soft', 'ware', '",', '\n', '   ', ' "', 'sub', 'category', '":', ' "', 'Product', 'ivity', ' Software', '",', '\n', '   ', ' "', 'platform', '":', ' "', 'Windows', '",', '\n', '   ', ' "', 'ed', 'ition', '":', ' "', 'Standard', '",', '\n', '   ']
input_str = 'Productivity Software'
positions = find_concat_positions_flexible(query['decoded_token'][0], input_str)
print(positions)  # 希望返回 [(16, 18)]

[]


In [189]:
# query['token'][0][120:].index(8684)
# query['decoded_token'][0][239:250]
query['text'][0]
# ''.join(query['decoded_token'][0])

' To enrich Entity 1 and Entity 2 with the required attributes, we first need to obtain the necessary information. I\'ll assume we have access to a database or API to fetch this data. Here\'s an example of how the output could look like:\n\n```json\n{\n  "Entity 1": {\n    "title": "Microsoft Visio Standard 2007 Version Upgrade",\n    "manufacturer": "Microsoft",\n    "price": 129.95,\n    "category": "Software",\n    "subcategory": "Productivity Software",\n    "platform": "Windows",\n    "edition": "Standard",\n    "type": "Upgrade",\n    "modelno": "VISSTAND2007"\n  },\n  "Entity 2": {\n    "title": "Adobe CS3 Design Standard Upgrade",\n    "manufacturer": "Adobe",\n    "price": 413.99,\n    "category": "Software",\n    "subcategory": "Design Software",\n    "platform": "Mac, Windows",\n    "edition": "Standard",\n    "type": "Upgrade",\n    "modelno": "CS3DESIGNUPG"\n  }\n}\n```\n\nKeep in mind that the provided information is just an example, and the actual values may vary dependi

In [118]:
def find_full_indices(decoded_token_list, target):
    # 将目标字符串拼接成单个字符串以便匹配
    target_concat = ''.join(target)
    n = len(decoded_token_list)
    for i in range(n):
        # 尝试从每个位置开始匹配整个目标字符串
        concat_str = ''
        for j in range(i, n):
            concat_str += decoded_token_list[j]
            if concat_str == target_concat:
                return (i, j)  # 返回开始和结束索引
    return (-1, -1)  # 如果没有找到，返回(-1, -1)

def find_positions(decoded_token_list, A, B, C, input_word):
    # 确定A, B, C的完整范围
    index_A_start, index_A_end = find_full_indices(decoded_token_list, A)
    index_B_start, index_B_end = find_full_indices(decoded_token_list, B)
    index_C_start, index_C_end = find_full_indices(decoded_token_list, C)
    
    # 如果找不到A, B, 或C
    if -1 in [index_A_start, index_B_start, index_C_start]:
        return []
    
    # 在B与C之间查找input_word
    positions = []
    input_word_concat = ''.join(input_word)
    concat_text = ''.join(decoded_token_list[index_B_end + 1:index_C_start + 1])
    start = 0
    while True:
        found_pos = concat_text.find(input_word_concat, start)
        if found_pos == -1:
            break
        positions.append(found_pos + index_B_end + 1)  # 转换为decoded_token_list中的位置
        start = found_pos + len(input_word_concat)
    
    return positions

# 示例数据
decoded_token_list = ["soft", "ware", "engineer", "develops", "soft", "ware"]
A = ["soft", "ware"]
B = ["develops"]
C = ["soft", "ware"]
input_word = ["soft"]

# 调用函数
positions = find_positions(decoded_token_list, A, B, C, input_word)
print("Positions of '", input_word, "' are: ", positions)

Positions of ' ['soft'] ' are:  []


In [90]:
tokenizer.encode('Entity 1')[1:],tokenizer.encode('Entity 2')[1:],tokenizer.encode('category'),tokenizer.encode('subcategory')[1:],tokenizer.encode('Software')[1:]

([20465, 28705, 28740], [20465, 28705, 28750], [1, 8011], [1083, 8684], [6583])

In [109]:
def find_matching_indexes(tokens, A_0, B, C, input_list):
    # 找到A_0的最后一个元素在tokens中的位置
    try:
        start_index = tokens.index(A_0[-1]) + 1
    except ValueError:
        # 如果A_0的最后一个元素不在tokens中，返回空列表
        return []

    # 初始化结果列表
    result_indexes = []

    # 初始化搜索区间的起始和结束位置
    start_search = start_index
    end_search = len(tokens)

    # 遍历tokens以查找B和C
    while start_search < end_search:
        try:
            # 在剩余tokens中寻找B
            start_b = tokens.index(B, start_search)
            # 在B之后寻找C
            end_c = tokens.index(C, start_b + len(B))
        except ValueError:
            # 如果找不到B或C，跳出循环
            break
        
        # 检查B和C之间是否有足够的空间匹配input_list
        if end_c - start_b >= len(input_list):
            # 遍历B和C之间的区域，寻找input_list
            for i in range(start_b + len(B), end_c - len(input_list) + 1):
                # 检查当前位置开始的序列是否与input_list匹配
                if tokens[i:i + len(input_list)] == input_list:
                    result_indexes.append(i)

        # 更新start_search为当前end_c之后，继续搜索下一个可能的位置
        start_search = end_c + len(C)

    return result_indexes
find_matching_indexes(query['token'][0],[20465, 28705, 28740],[8684], [1666, 8684],[6583])

[]

'sample_log_likelihoods'的sum是'sample_log_probs'
'logprobs'的sum是'cumulative_logprob'

In [31]:
# len(sum(stats['sample_log_likelihoods'][0],[]))
sum(stats['sample_log_likelihoods'][0][0]),stats['sample_log_probs'][0][0]

(-18.096956720289306, -18.096956720289306)

In [13]:
import numpy as np
from typing import Dict

def calculate_sentence_SAR(stats: Dict[str, np.ndarray], t: float = 0.001) -> np.ndarray:
    """
    Function to estimate the sequence-level uncertainty of a language model following the method of
    "Sentence SAR" as described in the paper https://arxiv.org/abs/2307.01379.
    This method calculates the sum of the probability of the generated text and text relevance relative to all other generations.

    Parameters:
        stats (Dict[str, np.ndarray]): input statistics, which for multiple samples includes:
            * corresponding log probabilities in 'sample_log_probs',
            * matrix with cross-encoder similarities in 'sample_sentence_similarity'
        t (float): A small threshold value to control the influence of relevance in uncertainty calculation.

    Returns:
        np.ndarray: float sentenceSAR for each sample in input statistics.
            Higher values indicate more uncertain samples.
    """
    batch_sample_log_probs = stats["sample_log_probs"]
    batch_sample_sentence_similarity = stats["sample_sentence_similarity"]

    sentenceSAR = []
    for sample_log_probs, sample_sentence_similarity in zip(
        batch_sample_log_probs, batch_sample_sentence_similarity
    ):
        sample_probs = np.exp(np.array(sample_log_probs))  # Convert log probabilities to regular probabilities
        R_s = (
            sample_probs
            * sample_sentence_similarity
            * (1 - np.eye(sample_sentence_similarity.shape[0]))  # Exclude self similarity
        )
        sent_relevance = R_s.sum(-1) / t
        E_s = -np.log(sent_relevance + sample_probs)
        sentenceSAR.append(E_s.mean())

    return np.array(sentenceSAR)
calculate_sentence_SAR(stats=result)
# Example usage:
# Assume 'stats' is a dictionary containing 'sample_log_probs' and 'sample_sentence_similarity'
# stats = {
#     "sample_log_probs": np.array([...]),
#     "sample_sentence_similarity": np.array([...])
# }
# result = calculate_sentence_SAR(stats)
# print(result)

array([0.50103766])

In [10]:
stats.keys()

dict_keys(['input_texts', 'target_texts', 'target_tokens', 'input_tokens', 'greedy_log_probs', 'greedy_tokens', 'greedy_tokens_alternatives', 'greedy_texts', 'greedy_log_likelihoods', 'embeddings_decoder', 'sample_log_likelihoods', 'sample_log_probs', 'sample_tokens', 'sample_texts', 'sample_sentence_similarity', 'sample_token_similarity', 'token_similarity'])

In [6]:
stats = np.load('/data/home/wangys/lm-polygraph/examples/sar.npy',allow_pickle=True).item()
batch_sample_log_likelihoods = stats["sample_log_likelihoods"]
batch_sample_token_similarity = stats["sample_token_similarity"]
batch_sample_sentence_similarity = stats["sample_sentence_similarity"]
t = 0.001
SAR = []
for batch_data in zip(
    batch_sample_log_likelihoods,
    batch_sample_token_similarity,
    batch_sample_sentence_similarity,
):
    sample_log_likelihoods = batch_data[0]
    sample_token_similarity = batch_data[1]
    sample_sentence_similarity = batch_data[2]

    tokenSAR = []
    for log_likelihoods, token_similarity in zip(
        sample_log_likelihoods, sample_token_similarity
    ):
        log_likelihoods = np.array(log_likelihoods)
        R_t = 1 - token_similarity
        R_t_norm = R_t / R_t.sum()
        E_t = -log_likelihoods * R_t_norm
        tokenSAR.append(E_t.sum())

    tokenSAR = np.array(tokenSAR)
    probs_token_sar = np.exp(-tokenSAR)
    R_s = (
        probs_token_sar
        * sample_sentence_similarity
        * (1 - np.eye(sample_sentence_similarity.shape[0]))
    )
    sent_relevance = R_s.sum(-1) / t
    E_s = -np.log(sent_relevance + probs_token_sar)
    SAR.append(E_s.mean())
np.array(SAR)

array([-8.35613904])

### Initialize model

In [2]:
base_model = AutoModelForCausalLM.from_pretrained(
    '/data/home/wangys/model/Mistral-7B-Instruct-v0.2',
    device_map='cuda:7',
)
tokenizer = AutoTokenizer.from_pretrained('/data/home/wangys/model/Mistral-7B-Instruct-v0.2', token_type_ids=None,
                clean_up_tokenization_spaces=False)

model = WhiteboxModel(base_model,tokenizer)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### Token level UE

In [12]:
estimator = SAR()
estimate_uncertainty(model, estimator, input_text='Who is George Bush?')

Some weights of the model checkpoint at /data/home/wangys/model/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
S

UncertaintyOutput(uncertainty=-8.356139042348568, input_text='Who is George Bush?', generation_text="George Bush may refer to one of two U.S. presidents named George Bush. Here's a brief overview of each:\n\n1. George H.W. Bush (b. 1924): He was the 41st President of the United States, serving from 1989 to 1993. Bush was a naval aviator in World War II and later served as a member of Congress from Texas. He also served as the", generation_tokens=[5163, 13668, 993, 3295, 298, 624, 302, 989, 500, 28723, 28735, 28723, 1258, 6640, 5160, 5163, 13668, 28723, 4003, 28742, 28713, 264, 6817, 23094, 302, 1430, 28747, 13, 13, 28740, 28723, 5163, 382, 28723, 28780, 28723, 13668, 325, 28726, 28723, 28705, 28740, 28774, 28750, 28781, 1329, 650, 403, 272, 28705, 28781, 28740, 303, 5120, 302, 272, 2969, 3543, 28725, 10732, 477, 28705, 28740, 28774, 28783, 28774, 298, 28705, 28740, 28774, 28774, 28770, 28723, 13668, 403, 264, 23850, 1182, 28710, 1028, 297, 3304, 3273, 3717, 304, 2062, 6117, 390, 264, 4

In [5]:
estimator = MaximumSequenceProbability()
estimate_uncertainty(model, estimator, input_text='Who is George Bush?')

Some weights of the model checkpoint at /data/home/wangys/model/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/wangys/anaconda3/envs/deepspeed/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue

UncertaintyOutput(uncertainty=13.410937309265137, input_text='Who is George Bush?', generation_text="George Bush may refer to one of two U.S. presidents named George Bush. Here's a brief overview of each:\n\n1. George H.W. Bush (b. 1924): He was the 41st President of the United States, serving from 1989 to 1993. Bush was a naval aviator in World War II and later served as a member of Congress from Texas. He also served as the", generation_tokens=[5163, 13668, 993, 3295, 298, 624, 302, 989, 500, 28723, 28735, 28723, 1258, 6640, 5160, 5163, 13668, 28723, 4003, 28742, 28713, 264, 6817, 23094, 302, 1430, 28747, 13, 13, 28740, 28723, 5163, 382, 28723, 28780, 28723, 13668, 325, 28726, 28723, 28705, 28740, 28774, 28750, 28781, 1329, 650, 403, 272, 28705, 28781, 28740, 303, 5120, 302, 272, 2969, 3543, 28725, 10732, 477, 28705, 28740, 28774, 28783, 28774, 298, 28705, 28740, 28774, 28774, 28770, 28723, 13668, 403, 264, 23850, 1182, 28710, 1028, 297, 3304, 3273, 3717, 304, 2062, 6117, 390, 264, 4

### Sequence level UE

In [ ]:
estimator = SAR()
estimate_uncertainty(model, estimator, input_text='Who is George Bush?')

In [11]:
estimator = MeanPointwiseMutualInformation()
estimate_uncertainty(model, estimator, input_text='Who is George Bush?')

Some weights of the model checkpoint at /data/home/wangys/model/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


UncertaintyOutput(uncertainty=-17.852901331445374, input_text='Who is George Bush?', generation_text="George Bush may refer to one of two U.S. presidents named George Bush. Here's a brief overview of each:\n\n1. George H.W. Bush (b. 1924): He was the 41st President of the United States, serving from 1989 to 1993. Bush was a naval aviator in World War II and later served as a member of Congress from Texas. He also served as the", generation_tokens=[5163, 13668, 993, 3295, 298, 624, 302, 989, 500, 28723, 28735, 28723, 1258, 6640, 5160, 5163, 13668, 28723, 4003, 28742, 28713, 264, 6817, 23094, 302, 1430, 28747, 13, 13, 28740, 28723, 5163, 382, 28723, 28780, 28723, 13668, 325, 28726, 28723, 28705, 28740, 28774, 28750, 28781, 1329, 650, 403, 272, 28705, 28781, 28740, 303, 5120, 302, 272, 2969, 3543, 28725, 10732, 477, 28705, 28740, 28774, 28783, 28774, 298, 28705, 28740, 28774, 28774, 28770, 28723, 13668, 403, 264, 23850, 1182, 28710, 1028, 297, 3304, 3273, 3717, 304, 2062, 6117, 390, 264, 

In [45]:
import numpy as np
from collections import defaultdict
from typing import List, Dict, Optional
stats = np.load('/data/home/wangys/lm-polygraph/examples/semantic_entropy.npy',allow_pickle=True).item()
def semantic_entropy(stats: Dict[str, np.ndarray]) -> np.ndarray:
    """
    Estimates the semantic entropy for each sample in the input statistics.

    Parameters:
        stats (Dict[str, np.ndarray]): input statistics, which for multiple samples includes:
            * generated samples in 'sample_texts',
            * corresponding log probabilities in 'sample_log_probs',
            * matrix with semantic similarities in 'semantic_matrix_entail'
    Returns:
        np.ndarray: float semantic entropy for each sample in input statistics.
            Higher values indicate more uncertain samples.
    """
    
    loglikelihoods_list = stats["sample_log_probs"]
    entailment_id = stats.get("entailment_id", 1)
    is_entailment = stats["semantic_matrix_entail"] == entailment_id
    print(is_entailment)

    # Concatenating hypotheses with input texts
    hyps_list = [[' '.join([input_text, hyp]) for hyp in stats["sample_texts"][i]] for i, input_text in enumerate(stats["input_texts"])]
    
    return batched_call(hyps_list, loglikelihoods_list, is_entailment)

def batched_call(hyps_list: List[List[str]], loglikelihoods_list: List[List[float]], is_entailment: np.ndarray, log_weights: Optional[List[List[float]]] = None) -> np.array:
    if log_weights is None:
        log_weights = [None] * len(hyps_list)
    
    semantic_logits = {}
    sample_to_class = {}
    class_to_sample = defaultdict(list)

    # Determine classes for hypotheses
    for idx, hyps in enumerate(hyps_list):
        sample_to_class[idx], class_to_sample[idx] = determine_classes(hyps, is_entailment[idx])

        # Collect likelihoods per class
        class_likelihoods = [np.array(loglikelihoods_list[idx])[np.array(class_idx)] for class_idx in class_to_sample[idx]]
        class_lp = [np.logaddexp.reduce(likelihoods) for likelihoods in class_likelihoods]
        
        # Apply weights if provided
        if log_weights[idx] is None:
            log_weights[idx] = [0] * len(hyps)
        
        semantic_logits[idx] = -np.mean([class_lp[sample_to_class[idx][j]] * np.exp(log_weights[idx][j]) for j in range(len(hyps))])
    
    return np.array([semantic_logits[i] for i in range(len(hyps_list))])

def determine_classes(hyps: List[str], is_entailment: np.ndarray) -> (Dict[int, int], Dict[int, List[int]]):
    sample_to_class = {}
    class_to_sample = defaultdict(list)

    for i in range(len(hyps)):
        if i == 0:
            class_to_sample[0] = [0]
            sample_to_class[0] = 0
            continue
        
        for class_id, class_indices in class_to_sample.items():
            class_text_id = class_indices[0]
            if is_entailment[class_text_id, i] and is_entailment[i, class_text_id]:
                class_to_sample[class_id].append(i)
                sample_to_class[i] = class_id
                break
        else:
            new_class_id = len(class_to_sample)
            class_to_sample[new_class_id] = [i]
            sample_to_class[i] = new_class_id
    
    return sample_to_class, class_to_sample

# Example usage:
# stats = {
#     'sample_log_probs': np.array([[0.1, 0.2], [0.4, 0.5]]),
#     'sample_texts': [['text1', 'text2'], ['text3', 'text4']],
#     'input_texts': ['input1', 'input2'],
#     'semantic_matrix_entail': np.array([[[1, 0], [0, 1]], [[1, 1], [1, 1]]]),
#     'entailment_id': 1
# }
output = semantic_entropy(stats)
output

[[[False False False False False False False False False False]
  [False False False False False False False False False False]
  [False False False False False False False False False False]
  [False False False False False False False False False False]
  [False False False False False False False False False False]
  [False False False False False False False False False False]
  [False False False False False False False False False False]
  [False False False False False False False False False False]
  [False False False False False False False False False False]
  [False False False False False False False False False False]]]


array([23.7672994])

In [44]:
stats["semantic_matrix_entail"]

array([[[0.99211544, 0.08175895, 0.4267827 , 0.80408055, 0.7590612 ,
         0.24637915, 0.09461544, 0.12501796, 0.97024375, 0.8976873 ],
        [0.74437773, 0.98674357, 0.11483058, 0.6593689 , 0.80901915,
         0.15827736, 0.04327307, 0.12684904, 0.57116574, 0.8476537 ],
        [0.3639897 , 0.01338555, 0.9920814 , 0.820661  , 0.14517976,
         0.01991121, 0.01533491, 0.01187618, 0.41261953, 0.17829414],
        [0.72689205, 0.1687846 , 0.37953278, 0.98945194, 0.93539363,
         0.6241864 , 0.20936836, 0.29448467, 0.96973497, 0.9535518 ],
        [0.67073053, 0.08700771, 0.00710694, 0.47592905, 0.99120855,
         0.02243401, 0.07285031, 0.12850712, 0.19683176, 0.22344266],
        [0.8139983 , 0.07905055, 0.04403748, 0.92957145, 0.5597203 ,
         0.9840152 , 0.07504941, 0.11253431, 0.893689  , 0.31243303],
        [0.35354203, 0.01060121, 0.00976979, 0.12878047, 0.6267275 ,
         0.02977836, 0.9878634 , 0.6174651 , 0.04306053, 0.13417588],
        [0.7687419 , 0.2255

In [6]:
estimator = LexicalSimilarity('rougeL')
estimate_uncertainty(model, estimator, input_text='Who is George Bush?')

Some weights of the model checkpoint at /data/home/wangys/model/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


UncertaintyOutput(uncertainty=-0.5354888443376555, input_text='Who is George Bush?', generation_text="George Bush may refer to one of two U.S. presidents named George Bush. Here's a brief overview of each:\n\n1. George H.W. Bush (b. 1924): He was the 41st President of the United States, serving from 1989 to 1993. Bush was a naval aviator in World War II and later served as a member of Congress from Texas. He also served as the", generation_tokens=[5163, 13668, 993, 3295, 298, 624, 302, 989, 500, 28723, 28735, 28723, 1258, 6640, 5160, 5163, 13668, 28723, 4003, 28742, 28713, 264, 6817, 23094, 302, 1430, 28747, 13, 13, 28740, 28723, 5163, 382, 28723, 28780, 28723, 13668, 325, 28726, 28723, 28705, 28740, 28774, 28750, 28781, 1329, 650, 403, 272, 28705, 28781, 28740, 303, 5120, 302, 272, 2969, 3543, 28725, 10732, 477, 28705, 28740, 28774, 28783, 28774, 298, 28705, 28740, 28774, 28774, 28770, 28723, 13668, 403, 264, 23850, 1182, 28710, 1028, 297, 3304, 3273, 3717, 304, 2062, 6117, 390, 264, 

In [8]:
import numpy as np
result = np.load('/data/home/wangys/lm-polygraph/examples/sent_sar_1726031992.7997897.npy',allow_pickle=True).item()
result.keys()

/home/wangys/anaconda3/envs/deepspeed/lib/python3.12/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


dict_keys(['input_texts', 'target_texts', 'target_tokens', 'input_tokens', 'greedy_log_probs', 'greedy_tokens', 'greedy_tokens_alternatives', 'greedy_texts', 'greedy_log_likelihoods', 'embeddings_decoder', 'sample_log_likelihoods', 'sample_log_probs', 'sample_tokens', 'sample_texts', 'sample_sentence_similarity', 'sample_token_similarity', 'token_similarity'])

In [28]:
result['greedy_texts']

['Barack Obama is a politician who served as the 44th President of the United States from January 20, 2009, to January 20, 2017. He was the first African American to hold the office. Obama was born on August 4, 1961, in Honolulu, Hawaii, and raised in Hawaii and Indonesia. He attended Columbia University and later earned his law degree from Harvard Law School.\n\nBefore']

In [36]:
result['sample_texts'][0][-3]

'Barack Obama was the 44th President of the United States, serving from January 20, 2009, to January 20, 2017. He was the first African American to hold the office. Obama was born on August 4, 1961, in Honolulu, Hawaii, and raised in Hawaii and Indonesia. He attended Columbia University in New York City and earned his law degree from Harvard Law School.\n\nBefore'

In [29]:
np.array(result['sample_log_likelihoods'][0]).sum(axis=1)

array([-15.41031588, -15.78380363,  -8.50700831, -16.47936298,
        -8.88130191, -12.57747051, -11.61657   ,  -6.4267327 ,
       -18.7933634 , -30.27003519])

In [6]:
estimator = SentenceSAR()
estimate_uncertainty(model, estimator, input_text='Who is Barack Obama?')

Some weights of the model checkpoint at /data/home/wangys/model/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
S

UncertaintyOutput(uncertainty=0.5010376612319298, input_text='Who is Barack Obama?', generation_text='Barack Obama is a politician who served as the 44th President of the United States from January 20, 2009, to January 20, 2017. He was the first African American to hold the office. Obama was born on August 4, 1961, in Honolulu, Hawaii, and raised in Hawaii and Indonesia. He attended Columbia University and later earned his law degree from Harvard Law School.\n\nBefore', generation_tokens=[3011, 468, 11764, 349, 264, 17587, 693, 6117, 390, 272, 28705, 28781, 28781, 362, 5120, 302, 272, 2969, 3543, 477, 4624, 28705, 28750, 28734, 28725, 28705, 28750, 28734, 28734, 28774, 28725, 298, 4624, 28705, 28750, 28734, 28725, 28705, 28750, 28734, 28740, 28787, 28723, 650, 403, 272, 907, 8623, 2556, 298, 2400, 272, 4007, 28723, 11764, 403, 5381, 356, 3628, 28705, 28781, 28725, 28705, 28740, 28774, 28784, 28740, 28725, 297, 7673, 328, 20366, 28725, 26434, 28725, 304, 6333, 297, 26434, 304, 24450, 28

In [ ]:
estimator = PointwiseMutualInformation()
estimate_uncertainty(model, estimator, input_text='Once upon a time there was a little girl who liked to')

In [4]:
ue_method = LexicalSimilarity()
input_text = "Who is George Bush?"
estimate_uncertainty(model, ue_method, input_text=input_text)

Some weights of the model checkpoint at /data/home/wangys/model/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


UncertaintyOutput(uncertainty=-0.5423855887497974, input_text='Who is George Bush?', generation_text="George Bush may refer to one of two U.S. presidents named George Bush. Here's a brief overview of each:\n\n1. George H.W. Bush (b. 1924): He was the 41st President of the United States, serving from 1989 to 1993. Bush was a naval aviator in World War II and later served as a member of Congress from Texas. He also served as the", generation_tokens=[5163, 13668, 993, 3295, 298, 624, 302, 989, 500, 28723, 28735, 28723, 1258, 6640, 5160, 5163, 13668, 28723, 4003, 28742, 28713, 264, 6817, 23094, 302, 1430, 28747, 13, 13, 28740, 28723, 5163, 382, 28723, 28780, 28723, 13668, 325, 28726, 28723, 28705, 28740, 28774, 28750, 28781, 1329, 650, 403, 272, 28705, 28781, 28740, 303, 5120, 302, 272, 2969, 3543, 28725, 10732, 477, 28705, 28740, 28774, 28783, 28774, 298, 28705, 28740, 28774, 28774, 28770, 28723, 13668, 403, 264, 23850, 1182, 28710, 1028, 297, 3304, 3273, 3717, 304, 2062, 6117, 390, 264, 

### BlackBox UE

In [ ]:
model = BlackboxModel(
    'YOUR_OPENAI_TOKEN',
    'gpt-3.5-turbo'
)
estimator = EigValLaplacian(verbose=True)
estimate_uncertainty(model, estimator, input_text='When did Albert Einstein die?')

In [ ]:
API_TOKEN = 'YOUR_API_TOKEN'
# for example let's take google/t5-small-ssm-nq model
MODEL_ID = 'google/t5-large-ssm-nqo'

model = BlackboxModel.from_huggingface(hf_api_token=API_TOKEN, hf_model_id=MODEL_ID, openai_api_key = None, openai_model_path = None)
ue_method = LexicalSimilarity()
input_text = "Who is George Bush?"
estimate_uncertainty(model, ue_method, input_text=input_text)

In [ ]:
# for example let's take bigscience/bloomz-560m model
MODEL_ID = 'bigscience/bloomz-560m'

model = BlackboxModel.from_huggingface(hf_api_token=API_TOKEN, hf_model_id=MODEL_ID, openai_api_key = None, openai_model_path = None)
ue_method = LexicalSimilarity()
input_text = "Who is George Bush?"
estimate_uncertainty(model, ue_method, input_text=input_text)